# Finetuning using IndoBART

In [1]:
! git clone https://github.com/indobenchmark/indonlg.git

Cloning into 'indonlg'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 152 (delta 75), reused 97 (delta 34), pack-reused 0
Receiving objects: 100% (152/152), 2.32 MiB | 6.62 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [2]:
%cd /content/indonlg
! pip install -r requirements.txt

/content/indonlg
     |████████████████████████████████| 1.5 MB 5.1 MB/s 
     |████████████████████████████████| 68 kB 6.9 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 186 kB 62.0 MB/s 
     |████████████████████████████████| 3.3 MB 51.6 MB/s 
     |████████████████████████████████| 394.7 MB 19 kB/s 
     |████████████████████████████████| 1.2 MB 56.0 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 65 kB 5.1 MB/s 
     |████████████████████████████████| 149 kB 78.1 MB/s 
     |████████████████████████████████| 1.8 MB 47.9 MB/s 
     |████████████████████████████████| 136 kB 68.8 MB/s 
     |████████████████████████████████| 212 kB 69.0 MB/s 
     |████████████████████████████████| 132 kB 77.3 MB/s 
     |████████████████████████████████| 462 kB 64.0 MB/s 
     |████████████████████████████████| 2.9 MB 40.1 MB/s 
     |████████████████████████████████| 14.8 MB 52.5 MB/s 
 

In [1]:
%cd /content/indonlg/examples

/content/indonlg/examples


In [2]:
import os, sys
sys.path.append('../')
os.chdir('../')

import torch
import shutil
import random
import numpy as np
import pandas as pd
from torch import optim
from transformers import MBartForConditionalGeneration

from indobenchmark import IndoNLGTokenizer
from utils.train_eval import train, evaluate
from utils.metrics import generation_metrics_fn
from utils.forward_fn import forward_generation
from utils.data_utils import MachineTranslationDataset, GenerationDataLoader

In [3]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
# Set random seed
# set_seed(26092020)

# Load Model

In [4]:
bart_model = MBartForConditionalGeneration.from_pretrained('indobenchmark/indobart-v2')
tokenizer = IndoNLGTokenizer.from_pretrained('indobenchmark/indobart-v2')

model = bart_model
model

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(40004, 768, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(40004, 768, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 768, padding_idx=1)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tru

In [5]:
count_param(model)

131543040

# Prepare Dataset

In [7]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = True
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[javanese]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = './save/MT_JAVNRF_INZNTV/example_id_jv'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [8]:
train_dataset_path = './dataset/MT_JAVNRF_INZNTV/train_preprocess.json'
valid_dataset_path = './dataset/MT_JAVNRF_INZNTV/valid_preprocess.json'
test_dataset_path = './dataset/MT_JAVNRF_INZNTV/test_preprocess.json'

train_dataset = MachineTranslationDataset(train_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset = MachineTranslationDataset(valid_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset = MachineTranslationDataset(test_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_loader = GenerationDataLoader(dataset=train_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

# Test model to generate sequences

In [9]:
inputs = ['aku pergi ke toko obat membeli <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[indonesian]', decoder_lang_token='[indonesian]')

bart_input.to(device)
bart_out = model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> aku pergi ke toko obat membeli <mask> [indonesian]
<s> aku pergi ke toko obat membeli obat.


In [10]:
inputs = ['aku menyang pasar karo <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[javanese]', decoder_lang_token='[javanese]')

bart_input.to(device)
bart_out = bart_model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> aku menyang pasar karo <mask> [javanese]
<s> aku menyang pasar karo tuku </s>


In [11]:
inputs = ['kuring ka pasar senen meuli daging <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[sundanese]', decoder_lang_token='[sundanese]')

bart_input.to(device)
bart_out = bart_model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> kuring ka pasar senen meuli daging <mask> [sundanese]
<s> kuring ka pasar senen meuli daging sapi.


# Test model to translate

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 150/150 [02:21<00:00,  1.06it/s]


In [14]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result.csv")

== Prediction Result ==
                                                 hyp  \
0  masa itu sampailah berita - berita tentang yes...   
1  dalam keadaan demikian, ketika aku dengan kuas...   
2  itu ia telah melihat ke depan dan telah berbic...   
3        karunia allah akan menyertai kamu sekalian.   
4  melakukan pekerjaan tangan yang berat. kalau k...   

                                               label  
0  nalika samana h ér od <0xC3> <0xA8> s, raja - ...  
1  " lan ing salebeting kawontenan ingkang makate...  
2  margi saking punika panjenenganipun uninga sad...  
3  sih - rah mat nung gila ing kowé kab <0xC3> <0...  
4  nganggo tanganku dhéwé aku nindak aké pagawé a...  

== Model Performance ==
           BLEU  SacreBLEU    ROUGE1    ROUGE2    ROUGEL  ROUGELsum
count  1.000000   1.000000  1.000000  1.000000  1.000000    1.00000
mean   1.271054   1.361085  7.638191  1.210668  7.449946    7.43584
std         NaN        NaN       NaN       NaN       NaN        NaN
min    1.271

In [15]:
metric_df

,BLEU,SacreBLEU,ROUGE1,ROUGE2,ROUGEL,ROUGELsum
0,1.271054,1.361085,7.638191,1.210668,7.449946,7.43584


# Fine Tuning & Evaluation

In [16]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.3223 LR:0.00010000: 100%|██████████| 746/746 [02:22<00:00,  5.22it/s]


(Epoch 1) TRAIN LOSS:2.3223 BLEU:39.39 SacreBLEU:41.01 ROUGE1:58.00 ROUGE2:31.57 ROUGEL:54.39 ROUGELsum:54.40 LR:0.00010000


VALID LOSS:1.8528: 100%|██████████| 100/100 [00:06<00:00, 15.98it/s]


(Epoch 1) VALID LOSS:1.8528 BLEU:38.73 SacreBLEU:38.93 ROUGE1:61.74 ROUGE2:36.27 ROUGEL:58.74 ROUGELsum:58.77


(Epoch 2) TRAIN LOSS:1.4764 LR:0.00009000: 100%|██████████| 746/746 [02:23<00:00,  5.21it/s]


...

(Epoch 9) VALID LOSS:1.9504 BLEU:45.78 SacreBLEU:45.97 ROUGE1:67.36 ROUGE2:43.94 ROUGEL:64.91 ROUGELsum:64.89


(Epoch 10) TRAIN LOSS:0.0983 LR:0.00003874: 100%|██████████| 746/746 [02:22<00:00,  5.22it/s]


(Epoch 10) TRAIN LOSS:0.0983 BLEU:95.20 SacreBLEU:95.34 ROUGE1:97.64 ROUGE2:95.37 ROUGEL:97.61 ROUGELsum:97.61 LR:0.00003874


VALID LOSS:1.9956: 100%|██████████| 100/100 [00:06<00:00, 16.09it/s]


(Epoch 10) VALID LOSS:1.9956 BLEU:46.06 SacreBLEU:46.21 ROUGE1:67.60 ROUGE2:44.32 ROUGEL:64.99 ROUGELsum:65.03


In [17]:
# Load best model
model.load_state_dict(torch.load(model_dir + "/best_model_0.th"))

<All keys matched successfully>

In [18]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 150/150 [02:43<00:00,  1.09s/it]


In [19]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_indobart_id-jv.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_id-jv.csv")

== Prediction Result ==
                                                 hyp  \
0  nalika samana paw arta bab yésus marang h ér o...   
1  " an ad éné ing kaanan mangkono iku, nalika ak...   
2  marga saka iku panjenengané wis pir sa man <0x...   
3  sih - rah mat nung gila ing kowé kab <0xC3> <0...   
4  wong - w ong mau padha nindak aké pagawé an ka...   

                                               label  
0  nalika samana h ér od <0xC3> <0xA8> s, raja - ...  
1  " lan ing salebeting kawontenan ingkang makate...  
2  margi saking punika panjenenganipun uninga sad...  
3  sih - rah mat nung gila ing kowé kab <0xC3> <0...  
4  nganggo tanganku dhéwé aku nindak aké pagawé a...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   36.099706  36.294524  56.893076  37.190303  52.974064  52.977945
std          NaN        NaN        NaN        NaN        NaN        

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
! cp './save/MT_JAVNRF_INZNTV/example_id_jv/best_model_0.th' '/content/drive/My Drive/'

In [26]:
! cp './save/MT_JAVNRF_INZNTV/example_id_jv/prediction_indobart_id-jv.csv' '/content/drive/My Drive/'